###Importing all the necessary libraries

In [ ]:
import numpy as np
import pandas as pd
import gensim
# upgrade gensim if you can't import softcossim
from gensim.matutils import softcossim 
from gensim import corpora
import gensim.downloader as api
from gensim.utils import simple_preprocess

print(gensim.__version__)
from datetime import datetime
import math
import requests
from io import BytesIO

3.6.0


In [ ]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
nltk.download('stopwords')
from sklearn.metrics.pairwise import cosine_similarity
!pip install fasttext
import fasttext
from numpy import dot
from numpy.linalg import norm
!pip install sister
!pip install fastparquet
import sister
fasttext_model300 = api.load('fasttext-wiki-news-subwords-300')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 68 kB 4.1 MB/s 
  Using cached pybind11-2.10.0-py3-none-any.whl (213 kB)
  Created wheel for fasttext: filename=fasttext-0.9.2-cp37-cp37m-linux_x86_64.whl size=3165320 sha256=5c8cad979f6c798f2c176a6e49c19cfa247bc3b0a6694ac4da7a9f250e3a100d
  Stored in directory: /root/.cache/pip/wheels/4e/ca/bf/b020d2be95f7641801a6597a29c8f4f19e38f9c02a345bab9b
Successfully built fasttext
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 19.7 MB 1.3 MB/s 
  Created wheel for sister: filename=sister-0.1.10-py3-none-any.whl size=5395 sha256=edd7c0d0c3d275195d22540e1d1ab928fb136814d08d14c6a813fcfbf9d9d5b4
  Stored in directory: /root/.cache/pip/wheels/8f/2d/fe/f78c1b809e161ae840ebaf3f4a7b95350a300f89d439718042
Successfully built sister
Looking in indexes: https://pypi.org/simple, h

In [ ]:
# Initializing the sentence embedding model.
sentence_embedding = sister.MeanEmbedding(lang="en")

Loading model...


####Reading the data and removing unecessary columns and stopwords. 

In [9]:
# Reading the data and removing unnecessary columns
article_prod = pd.read_parquet('/content/QUINTYPE_ARTICLE_PROD.parquet', engine='fastparquet')
article_prod = article_prod.drop_duplicates(subset='id', keep="first")
article_prod = article_prod.drop('mid', axis=1)
article_prod = article_prod.drop('slug', axis=1)
article_prod = article_prod.drop('imageUrl', axis=1)
article_prod = article_prod.drop('metaDescription', axis=1)
article_prod = article_prod.drop('offset', axis=1)
article_prod = article_prod.drop('partition', axis=1)
article_prod = article_prod.drop('processTimestamp', axis=1)
article_prod['qid_aux'] = np.arange(len(article_prod))

# Removing stopwords
stop_words = set(stopwords.words('english'))
article_prod['headline'] = article_prod['headline'].apply(lambda words: ' '.join(word.lower() for word in words.split() if word not in stop_words))

# Adding the sentence embeddings to the dataset

article_prod
# headline cos sim
# section_name fuzzy match partial matching ratio
# tagname fasttext vector.
# date diff

,id,headline,text,authorName,publishedAt,firstPublishedAt,sectionName,tagName,contentType,updatedTimestamp,date,hour,qid_aux
0,3ef8a5d6-b0ac-4be0-b33e-691778a1d846,neetu singh's emotional heartspeak rishi kapoo...,It’s his 69th birth anniversary tomorrow (4 Se...,Khalid Mohamed,1630681541612,1630681541612,Bollywood:Celebrities:Entertainment,"Neetu Kapoor,Rishi Kapoor birth anniversary,",story,1636880016750,2021-09-03,15,0
2,0f14e512-f669-40cf-81bf-69f5f74160ec,abp-cvoter: bjp see drop seats but rise past m...,"The ABP-CVoter survey on Friday, 3 September, ...",Mekhala Saran,1630691925314,1630676169111,Elections:Hot-News:India:News:Politics,"uttar pradesh elections 2022,",story,1636880016757,2021-09-03,17,1
6,629e3dd6-3a75-47be-a7a2-ddb44c6a757d,kamala harris’ early years at school through ‘...,"Carole Porter lived in a little yellow house, ...",Savita Patel,1630687647145,1630687647145,The-Indian-American,"Childhood,Kamala Harris,US Vice President,",story,1636880017076,2021-09-03,16,2
9,2d10e424-a270-4ad9-bb01-9d5177daf08f,in photos: india this week,From Sumit Antil winning a Gold at the Tokyo p...,The Quint,1630692067708,1630678724037,Hot-News:India:Photos,"India This Week,Photos,photo story,",story,1636880017077,2021-09-03,18,3
12,cee1a20b-0961-4ef6-9c77-a87e3daf8da0,clip man stealing electricity pak shared with ...,A video showing a man boasting about stealing ...,Abhilash Mallick,1630680066960,1630680066960,WebQoof,"Pakistan,Karachi,Electricity,Webqoof,Stealing,...",story,1636880017182,2021-09-03,14,4
14,cd4dfe5c-6f65-46e2-a54a-4f9f1b532403,"as protests intensify, skm announces 'biggest ...",After Karnal Sub-Divisional Magistrate (SDM) A...,The Quint,1630680754220,1630680754220,Hot-News:India,"BJP Government,farmers protests,",story,1636880017183,2021-09-03,14,5
17,47650489-eef0-41c4-ab8a-8ea21447d13a,"amid ronaldo’s many achievements, serious rape...",From Fans to Rape ApologistsThere is no denyin...,Hazel Gandhi,1630665017573,1630665017573,Explainers:Hot-Take:NEON:Sports:Sports-Buzz,"Sexual Harassment,Football,Cristiano Ronaldo,R...",story,1636880017184,2021-09-03,15,6
21,348dd529-a6f9-4d07-9e73-3d9b954252ee,avani lekhara: wasn't satisfied with my gold o...,Shooter Avani Lekhara said on Friday that a go...,Mendra Dorjey Sahni,1630695751434,1630695751434,India:News-Videos:Olympic-Sports:Sports,"Tokyo Paralympics 2020,Avani Lekhara,",story,1636880017185,2021-09-03,19,7
23,e2367243-eb41-4f5d-91aa-7db0b907be72,abp-cvoter poll: aap may get 55 seats punjab t...,The ABP-CVoter survey predicted that the Aam A...,Aditya Menon,1630700341034,1630676410581,Elections:Politics,"Captain Amarinder Singh,AAP Punjab,Punjab Cong...",story,1636880017186,2021-09-03,20,8
26,2a1dc184-f7ba-4f3e-ae32-d634777b85f6,abp cvoter poll: harish rawat emerges preferre...,"Ahead of the Uttarakhand Assembly elections, s...",The Quint,1630679640934,1630676894075,Hot-News:Politics,"BJP,Congress,Harish Rawat,uttarakhand assembly...",story,1636880017433,2021-09-03,14,9


In [10]:

documents = article_prod['headline'].tolist()
dictionary = corpora.Dictionary([simple_preprocess(doc) for doc in documents])

# Prepare the similarity matrix
similarity_matrix= fasttext_model300.similarity_matrix(dictionary, tfidf=None, threshold=0.0, exponent=2.0, nonzero_limit=100)

# Convert the sentences into bag-of-words vectors.
sentences=[]

for doc in documents:
  sentences.append(dictionary.doc2bow(simple_preprocess(doc)))
print(sentences)  



[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1)], [(10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 1), (19, 1), (20, 1), (21, 1), (22, 1)], [(23, 1), (24, 1), (25, 1), (26, 1), (27, 1), (28, 1), (29, 1), (30, 1), (31, 1), (32, 1), (33, 1)], [(34, 1), (35, 1), (36, 1), (37, 1), (38, 1)], [(35, 1), (39, 1), (40, 1), (41, 1), (42, 1), (43, 1), (44, 1), (45, 1), (46, 1), (47, 1)], [(48, 1), (49, 1), (50, 1), (51, 1), (52, 1), (53, 1), (54, 1), (55, 1), (56, 1)], [(57, 1), (58, 1), (59, 1), (60, 1), (61, 1), (62, 1), (63, 1), (64, 1), (65, 1), (66, 1), (67, 1)], [(47, 1), (68, 1), (69, 1), (70, 1), (71, 1), (72, 1), (73, 1), (74, 1), (75, 1), (76, 1), (77, 1)], [(10, 1), (13, 1), (20, 1), (78, 1), (79, 1), (80, 1), (81, 1), (82, 1), (83, 1), (84, 1), (85, 1), (86, 1)], [(10, 1), (13, 1), (83, 1), (87, 1), (88, 1), (89, 1), (90, 1), (91, 1), (92, 1), (93, 1)], [(94, 1), (95, 1), (96, 1), (97, 1), (98, 1), (99, 1)], [(100, 1), (101

In [11]:
def create_soft_cossim_matrix(sentences):
    len_array = np.arange(len(sentences))
    xx, yy = np.meshgrid(len_array, len_array)
    cossim_mat = pd.DataFrame([[round(softcossim(sentences[i],sentences[j], similarity_matrix) ,6) for i, j in zip(x,y)] for y, x in zip(xx, yy)])
    return cossim_mat

mat=create_soft_cossim_matrix(sentences)
mat

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,1.000000,0.000000,0.000000,0.028850,0.000000,0.000000,0.000000,0.000000,0.000000,0.038899,0.000000,0.000000,0.029824,0.000000,0.062563
1,0.000000,1.000000,0.147067,0.000000,0.029058,0.031316,0.032029,0.059233,0.329660,0.227888,0.000000,0.033559,0.056206,0.062715,0.000000
2,0.000000,0.147067,1.000000,0.252908,0.151417,0.000000,0.041280,0.147825,0.000000,0.000000,0.044275,0.000000,0.158953,0.143926,0.000000
3,0.028850,0.000000,0.252908,1.000000,0.184844,0.000000,0.043253,0.098984,0.143336,0.000000,0.076494,0.000000,0.114736,0.065598,0.261808
4,0.000000,0.029058,0.151417,0.184844,1.000000,0.000000,0.036962,0.095346,0.039930,0.000000,0.000000,0.000000,0.000000,0.040037,0.131414
5,0.000000,0.031316,0.000000,0.000000,0.000000,1.000000,0.000000,0.034816,0.042887,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,0.000000,0.032029,0.041280,0.043253,0.036962,0.000000,1.000000,0.076395,0.029582,0.000000,0.055428,0.000000,0.032025,0.000000,0.034752
7,0.000000,0.059233,0.147825,0.098984,0.095346,0.034816,0.076395,1.000000,0.072619,0.035985,0.000000,0.043261,0.000000,0.038174,0.000000
8,0.000000,0.329660,0.000000,0.143336,0.039930,0.042887,0.029582,0.072619,1.000000,0.263706,0.070492,0.054522,0.000000,0.000000,0.072264
9,0.038899,0.227888,0.000000,0.000000,0.000000,0.000000,0.000000,0.035985,0.263706,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000


##### Creating a dataset that contains qid, and grade on the basis of the cosine similarity of the sentence vector embeddings.

In [12]:
from nltk.util import re_show
# Creating a dataset that contains qid, and grade on the basis of the cosine similarity of the sentence vector embeddings.
n = article_prod.shape[0]

# Create our output matrix


# Compute cosine similarity matrix


# Creating new dataset
test_df = pd.DataFrame(columns = ['qid', 'id', ])
#row
#ol
mat_time=[[0 for x in range(n)] for y in range(n)] 
time_diff = []
for i in range(n):
    for j in range(n):
        if(i==j): continue;
        res=(datetime.strptime(article_prod['date'].values[i], "%Y-%m-%d")-datetime.strptime(article_prod['date'].values[j], "%Y-%m-%d")).days
        mat_time[i][j]=res
#test_df['time_diff']=time_diff
# Maybe we should use BM25 instead? 


# Populating new dataset with qid, id and the cosine_similarity.


#list(test_df['time_diff'])
# Once we have the dataset, we can assign a grade to each by looping and on the basis of the cosine_similarity
# 0  < 0.1175  < 0.2212 < 0.393 < 0.632 < 0.865 < 1
# 0  <   1     <   2    <   3   <   4   <  5  


# We now have the graded dataset... We need to find feature vectors and add them to the dataset.
# We'll start with the following.


1. Time
2. Similarity Score



###Vectorizing the tag name and creating a soft cosine similarity matrix 

In [13]:
article_prod['tagName'] = article_prod['tagName'].apply(lambda words: ' '.join(word.lower() for word in words.split() if word not in stop_words))

documents = article_prod['tagName'].tolist()
dictionary = corpora.Dictionary([simple_preprocess(doc) for doc in documents])

# Prepare the similarity matrix
similarity_matrix= fasttext_model300.similarity_matrix(dictionary, tfidf=None, threshold=0.0, exponent=2.0, nonzero_limit=100)

# Convert the sentences into bag-of-words vectors.
sentences_tag=[]

for doc in documents:
  sentences_tag.append(dictionary.doc2bow(simple_preprocess(doc)))
print(sentences_tag)  




[[(0, 1), (1, 1), (2, 2), (3, 1), (4, 1)], [(5, 1), (6, 1), (7, 1)], [(8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1)], [(14, 1), (15, 1), (16, 1), (17, 1), (18, 1), (19, 1)], [(20, 1), (21, 1), (22, 1), (23, 1), (24, 1), (25, 1), (26, 1)], [(27, 1), (28, 1), (29, 1), (30, 1)], [(31, 1), (32, 1), (33, 1), (34, 1), (35, 1), (36, 1), (37, 1)], [(38, 1), (39, 1), (40, 1), (41, 1)], [(42, 1), (43, 1), (44, 1), (45, 1), (46, 1), (47, 1), (48, 1), (49, 1), (50, 2), (51, 2), (52, 5), (53, 1), (54, 1), (55, 1)], [(27, 1), (46, 1), (56, 1), (57, 1), (58, 1), (59, 1), (60, 1)], [(61, 1), (62, 1), (63, 1), (64, 2)], [(65, 1), (66, 1), (67, 1), (68, 1), (69, 1), (70, 1)], [(17, 1), (71, 2), (72, 1), (73, 3), (74, 1), (75, 2), (76, 1)], [(77, 1), (78, 1), (79, 1), (80, 1), (81, 1), (82, 1)], [(83, 1), (84, 1), (85, 1)]]


In [15]:
def create_soft_cossim_matrix(sentences_tag):
    len_array = np.arange(len(sentences_tag))
    xx, yy = np.meshgrid(len_array, len_array)
    cossim_mat = pd.DataFrame([[round(softcossim(sentences_tag[i],sentences_tag[j], similarity_matrix) ,6) for i, j in zip(x,y)] for y, x in zip(xx, yy)])
    return cossim_mat

mat_tag=create_soft_cossim_matrix(sentences_tag)
mat_tag

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,1.000000,0.0,0.083636,0.033908,0.000000,0.000000,0.071056,0.041295,0.000000,0.000000,0.000000,0.0000,0.162640,0.000000,0.0
1,0.000000,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.0
2,0.083636,0.0,1.000000,0.114807,0.158935,0.000000,0.000000,0.071647,0.017257,0.000000,0.000000,0.0000,0.069830,0.000000,0.0
3,0.033908,0.0,0.114807,1.000000,0.086311,0.000000,0.000000,0.000000,0.078800,0.000000,0.000000,0.0000,0.208289,0.035826,0.0
4,0.000000,0.0,0.158935,0.086311,1.000000,0.000000,0.088854,0.000000,0.063642,0.049294,0.000000,0.0000,0.121593,0.032367,0.0
5,0.000000,0.0,0.000000,0.000000,0.000000,1.000000,0.047670,0.186329,0.000000,0.188982,0.000000,0.0000,0.000000,0.000000,0.0
6,0.071056,0.0,0.000000,0.000000,0.088854,0.047670,1.000000,0.128247,0.015710,0.036035,0.000000,0.0512,0.000000,0.126161,0.0
7,0.041295,0.0,0.071647,0.000000,0.000000,0.186329,0.128247,1.000000,0.038007,0.140851,0.000000,0.0000,0.000000,0.000000,0.0
8,0.000000,0.0,0.017257,0.078800,0.063642,0.000000,0.015710,0.038007,1.000000,0.100996,0.000000,0.0000,0.043981,0.000000,0.0
9,0.000000,0.0,0.000000,0.000000,0.049294,0.188982,0.036035,0.140851,0.100996,1.000000,0.054178,0.0000,0.000000,0.000000,0.0


###Vectorizing the text and creating a soft cosine similarity matrix 

In [14]:
article_prod['text'] = article_prod['text'].apply(lambda words: ' '.join(word.lower() for word in words.split() if word not in stop_words))

documents = article_prod['text'].tolist()
dictionary = corpora.Dictionary([simple_preprocess(doc) for doc in documents])

# Prepare the similarity matrix
similarity_matrix= fasttext_model300.similarity_matrix(dictionary, tfidf=None, threshold=0.0, exponent=2.0, nonzero_limit=100)

# Convert the sentences into bag-of-words vectors.
sentences_text=[]

for doc in documents:
  sentences_text.append(dictionary.doc2bow(simple_preprocess(doc)))
print(sentences_text)  


[[(0, 1), (1, 1), (2, 1), (3, 2), (4, 1), (5, 1), (6, 1), (7, 1), (8, 2), (9, 2), (10, 1), (11, 1), (12, 1), (13, 1), (14, 2), (15, 1), (16, 1), (17, 7), (18, 1), (19, 1), (20, 1), (21, 10), (22, 1), (23, 1), (24, 1), (25, 1), (26, 1), (27, 1), (28, 1), (29, 1), (30, 1), (31, 3), (32, 1), (33, 1), (34, 1), (35, 1), (36, 1), (37, 1), (38, 1), (39, 4), (40, 1), (41, 1), (42, 1), (43, 3), (44, 3), (45, 1), (46, 1), (47, 1), (48, 1), (49, 1), (50, 5), (51, 1), (52, 3), (53, 1), (54, 1), (55, 8), (56, 3), (57, 1), (58, 1), (59, 1), (60, 6), (61, 1), (62, 1), (63, 1), (64, 1), (65, 2), (66, 1), (67, 1), (68, 1), (69, 1), (70, 1), (71, 1), (72, 1), (73, 1), (74, 2), (75, 5), (76, 2), (77, 2), (78, 4), (79, 1), (80, 1), (81, 1), (82, 1), (83, 1), (84, 1), (85, 1), (86, 1), (87, 2), (88, 1), (89, 1), (90, 2), (91, 2), (92, 1), (93, 1), (94, 1), (95, 1), (96, 1), (97, 7), (98, 1), (99, 1), (100, 1), (101, 1), (102, 1), (103, 1), (104, 1), (105, 1), (106, 1), (107, 2), (108, 1), (109, 2), (110, 1

In [16]:
def create_soft_cossim_matrix(sentences_text):
    len_array = np.arange(len(sentences_text))
    xx, yy = np.meshgrid(len_array, len_array)
    cossim_mat = pd.DataFrame([[round(softcossim(sentences_text[i],sentences_text[j], similarity_matrix) ,6) for i, j in zip(x,y)] for y, x in zip(xx, yy)])
    return cossim_mat

mat_text=create_soft_cossim_matrix(sentences_text)
mat_text

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,1.000000,0.416857,0.542238,0.147512,0.344080,0.330968,0.490125,0.445206,0.312436,0.209650,0.298516,0.318866,0.234276,0.245037,0.375034
1,0.416857,1.000000,0.295531,0.166182,0.243631,0.318071,0.349601,0.271851,0.510117,0.588048,0.226870,0.247120,0.158515,0.192213,0.198571
2,0.542238,0.295530,1.000000,0.107952,0.258847,0.236588,0.375184,0.331114,0.251131,0.191835,0.214137,0.214760,0.213021,0.218917,0.290522
3,0.147512,0.166182,0.107952,1.000000,0.153620,0.132338,0.154459,0.237535,0.165780,0.109797,0.116355,0.099534,0.155902,0.079662,0.160486
4,0.344080,0.243631,0.258847,0.153620,1.000000,0.236075,0.387034,0.245826,0.256361,0.187089,0.237278,0.248411,0.213704,0.323957,0.185864
5,0.330967,0.318071,0.236588,0.132338,0.236075,1.000000,0.341492,0.249422,0.339284,0.280845,0.197819,0.265412,0.193539,0.248046,0.215604
6,0.490125,0.349601,0.375184,0.154459,0.387034,0.341492,1.000000,0.382838,0.313764,0.238285,0.262526,0.358818,0.222176,0.278432,0.242457
7,0.445206,0.271851,0.331114,0.237535,0.245826,0.249422,0.382838,1.000000,0.261111,0.188033,0.230507,0.214951,0.175065,0.170473,0.261855
8,0.312436,0.510117,0.251131,0.165780,0.256361,0.339284,0.313764,0.261111,1.000000,0.605527,0.206326,0.278715,0.198359,0.204632,0.223277
9,0.209650,0.588048,0.191835,0.109797,0.187089,0.280845,0.238285,0.188033,0.605527,1.000000,0.130431,0.231220,0.160194,0.180289,0.135183


####Creating the final dataset

In [ ]:
#creating the final dataset
def findGrade(num):
  if(0.00 <= num and num < 0.1175 ):
     return 0
  elif (0.1175 < num and num < 0.2212):
     return 1
  elif(0.2212 < num and num < 0.393):
     return 2
  elif(0.393< num and num < 0.632  ):
      return 3
  elif(0.632 < num and num < 0.865):
      return 4
  else:
      return 5           
for i in range(n):
    for j in range(n):
        if(i==j): continue;
        tmp = {'qid': article_prod['id'].values[i], 'id': article_prod['id'].values[j], 'grade': findGrade(mat[i][j]),'tag_sim':mat_tag[i][j],'text_sim':mat_text[i][j],'timeDiff':math.exp(-mat_time[i][j])}
        test_df = test_df.append(tmp,ignore_index= True)
test_df['features'] = [', '.join(str(x) for x in y) for y in map(tuple, test_df[['tag_sim', 'text_sim','timeDiff']].values)]
for i in test_df['features']:
  i=i.split(",")
  for c in i:
    i.append(float(c))
  print(i)  
  

test_df['group_id']=test_df.groupby(['qid'])['id'].ngroup()



#pd.set_option( "display.max_rows", None)
#test_df

In [ ]:
lambdas_per_query = test_df.copy()
"""
for i in lambdas_per_query['features']:
  for j in i:
    float(j)
  i.tolist()"""
lambdas_per_query['last_prediction'] = 0.0

lambdas_per_query.sort_values(['group_id', 'last_prediction'], ascending=[True, False], kind='stable')
lambdas_per_query['display_rank'] = lambdas_per_query.groupby('qid').cumcount()

lambdas_per_query['discount'] = 1 / np.log2(2 + lambdas_per_query['display_rank'])
lambdas_per_query['gain'] = (2**lambdas_per_query['grade'] - 1)

lambdas_per_query[['qid', 'display_rank', 'discount', 'grade', 'gain','features']]



,qid,display_rank,discount,grade,gain,features
0,3ef8a5d6-b0ac-4be0-b33e-691778a1d846,0,1.000000,0.0,0.0,"0.315019, 0.416857, 1.0"
1,3ef8a5d6-b0ac-4be0-b33e-691778a1d846,1,0.630930,0.0,0.0,"0.366067, 0.542238, 1.0"
2,3ef8a5d6-b0ac-4be0-b33e-691778a1d846,2,0.500000,0.0,0.0,"0.29459, 0.147512, 1.0"
3,3ef8a5d6-b0ac-4be0-b33e-691778a1d846,3,0.430677,0.0,0.0,"0.248749, 0.34408, 1.0"
4,3ef8a5d6-b0ac-4be0-b33e-691778a1d846,4,0.386853,0.0,0.0,"0.159258, 0.330967, 1.0"
5,3ef8a5d6-b0ac-4be0-b33e-691778a1d846,5,0.356207,0.0,0.0,"0.20161, 0.490125, 1.0"
6,3ef8a5d6-b0ac-4be0-b33e-691778a1d846,6,0.333333,0.0,0.0,"0.098483, 0.445206, 1.0"
7,3ef8a5d6-b0ac-4be0-b33e-691778a1d846,7,0.315465,0.0,0.0,"0.372187, 0.312436, 1.0"
8,3ef8a5d6-b0ac-4be0-b33e-691778a1d846,8,0.301030,0.0,0.0,"0.394693, 0.20965, 1.0"
9,3ef8a5d6-b0ac-4be0-b33e-691778a1d846,9,0.289065,0.0,0.0,"0.12674, 0.298516, 1.0"


In [ ]:
# each group paired with each other group
swaps = lambdas_per_query.merge(lambdas_per_query, on='group_id', how='outer')
# changes[j][i] = changes[i][j] = (discount(i) - discount(j)) * (gain(rel[i]) - gain(rel[j]));
swaps['delta'] = np.abs((swaps['discount_x'] - swaps['discount_y']) * (swaps['gain_x'] - swaps['gain_y']))
swaps[['group_id', 'display_rank_x', 'display_rank_y', 'delta']]


In [ ]:
swaps['rho'] = 1 / (1 + np.exp(swaps['last_prediction_x'] - swaps['last_prediction_y']))
swaps[['group_id', 'display_rank_x', 'display_rank_y', 'delta', 'last_prediction_x', 'last_prediction_y', 'rho']]

In [ ]:
swaps['lambda'] = 0
slice_x_better =swaps[swaps['grade_x'] > swaps['grade_y']]
swaps.loc[swaps['grade_x'] > swaps['grade_y'], 'lambda'] = slice_x_better['delta'] * slice_x_better['rho']
swaps[['group_id', 'display_rank_x', 'display_rank_y', 'delta', 'last_prediction_x', 'last_prediction_y', 'rho', 'lambda']]

In [ ]:
# Better minus worse
lambdas_x = swaps.groupby(['group_id', 'display_rank_x'])['lambda'].sum().rename('lambda')
lambdas_y = swaps.groupby(['group_id', 'display_rank_y'])['lambda'].sum().rename('lambda')
lambdas = lambdas_x - lambdas_y
lambdas
lambdas_per_query = lambdas_per_query.merge(lambdas, left_on=['group_id', 'display_rank'], right_on=['group_id', 'display_rank_x'], how='left')
lambdas_per_query[['group_id', 'id', 'grade', 'features', 'lambda']]

,group_id,id,grade,features,lambda
0,4,0f14e512-f669-40cf-81bf-69f5f74160ec,0.0,"0.315019, 0.416857, 1.0",0.000000
1,4,629e3dd6-3a75-47be-a7a2-ddb44c6a757d,0.0,"0.366067, 0.542238, 1.0",0.000000
2,4,2d10e424-a270-4ad9-bb01-9d5177daf08f,0.0,"0.29459, 0.147512, 1.0",0.000000
3,4,cee1a20b-0961-4ef6-9c77-a87e3daf8da0,0.0,"0.248749, 0.34408, 1.0",0.000000
4,4,cd4dfe5c-6f65-46e2-a54a-4f9f1b532403,0.0,"0.159258, 0.330967, 1.0",0.000000
5,4,47650489-eef0-41c4-ab8a-8ea21447d13a,0.0,"0.20161, 0.490125, 1.0",0.000000
6,4,348dd529-a6f9-4d07-9e73-3d9b954252ee,0.0,"0.098483, 0.445206, 1.0",0.000000
7,4,e2367243-eb41-4f5d-91aa-7db0b907be72,0.0,"0.372187, 0.312436, 1.0",0.000000
8,4,2a1dc184-f7ba-4f3e-ae32-d634777b85f6,0.0,"0.394693, 0.20965, 1.0",0.000000
9,4,c9db5fad-3428-4a62-bf3e-65f1f962b7a6,0.0,"0.12674, 0.298516, 1.0",0.000000


In [ ]:
from sklearn.tree import DecisionTreeRegressor, plot_tree

#2. Train a regression tree on this round's lambdas
features = lambdas_per_query['features'].tolist()
tree = DecisionTreeRegressor(max_leaf_nodes=10)
tree.fit(features, lambdas_per_query['lambda'])    

tree

ValueError: ignored

In [ ]:

def compute_lambdas(lambdas_per_query):
    lambdas_per_query = lambdas_per_query.sort_values(['group_id', 'last_prediction'], ascending=[True, False], kind='stable')
    lambdas_per_query['display_rank'] = lambdas_per_query.groupby('group_id').cumcount()

    #TBD - How do generalize this to any metric?
    lambdas_per_query['discount'] = 1 / np.log2(2 + lambdas_per_query['display_rank'])
    lambdas_per_query['gain'] = (2**lambdas_per_query['grade'] - 1)

    # swaps dataframe holds each pair-wise swap computed (shrink columns for memory?)   
    # Optimization of swaps = lambdas_per_query.merge(lambdas_per_query, on='qid', how='outer')
    # to limit to just needed columns
    to_swap = lambdas_per_query[['group_id', 'display_rank', 'grade', 'last_prediction', 'discount', 'gain']]
    #to_swap = lambdas_per_query
    swaps = to_swap.merge(to_swap, on='group_id', how='outer')

    # delta - delta in DCG due to swap
    swaps['delta'] = np.abs((swaps['discount_x'] - swaps['discount_y']) * (swaps['gain_x'] - swaps['gain_y']))
    
    # rho - based on current model prediction delta
    swaps['rho'] = 1 / (1 + np.exp(swaps['last_prediction_x'] - swaps['last_prediction_y']))
    
    # If you want to be pure gradient boosting, weight reweights each models prediction
    # I haven't found this to matter in practice
    swaps['weight'] = swaps['rho'] * (1.0 - swaps['rho']) * swaps['delta']

    # Compute lambdas (the next model in ensemble's predictors) when grade_x > grade_y
    swaps['lambda'] = 0
    slice_x_better =swaps[swaps['grade_x'] > swaps['grade_y']]
    swaps.loc[swaps['grade_x'] > swaps['grade_y'], 'lambda'] = slice_x_better['delta'] * slice_x_better['rho']
    
    # accumulate lambdas and add back to model
    lambdas_x = swaps.groupby(['group_id', 'display_rank_x'])['lambda'].sum().rename('lambda')
    lambdas_y = swaps.groupby(['group_id', 'display_rank_y'])['lambda'].sum().rename('lambda')

    weights_x = swaps.groupby(['group_id', 'display_rank_x'])['weight'].sum().rename('weight')
    weights_y = swaps.groupby(['group_id', 'display_rank_y'])['weight'].sum().rename('weight')
    
    weights = weights_x + weights_y
    lambdas = lambdas_x - lambdas_y

    lambdas_per_query = lambdas_per_query.merge(lambdas, 
                                                left_on=['group_id', 'display_rank'], 
                                                right_on=['group_id', 'display_rank_x'], 
                                                how='left')
    lambdas_per_query = lambdas_per_query.merge(weights, 
                                                left_on=['group_id', 'display_rank'], 
                                                right_on=['group_id', 'display_rank_x'], 
                                                how='left')

    return lambdas_per_query

In [ ]:
lambdas_per_query['lambda'][1]

0.0

In [ ]:
ensemble=[]
def lambda_mart_pure(judgments, rounds=20,
                     learning_rate=0.1, max_leaf_nodes=8):

    lambdas_per_query = judgments.copy()
    lambdas_per_query['last_prediction'] = 0.0

    for i in range(0, rounds):
        print(f"round {i}")

        # ------------------
        #1. Build pair-wise predictors for this round
        lambdas_per_query = compute_lambdas(lambdas_per_query)

        # ------------------
        #2. Train a regression tree on this round's lambdas
        features = lambdas_per_query['features'].tolist()
        new_tree = DecisionTreeRegressor(max_leaf_nodes=max_leaf_nodes)
        new_tree.fit(features, lambdas_per_query['lambda'])    

        # -------------------
        #4. Add to ensemble, recreate last prediction
        ensemble.append(new_tree)
        next_predictions = new_tree.predict(features)
        lambdas_per_query['last_prediction'] += (next_predictions * learning_rate) 
        
        print(lambdas_per_query.loc[0, ['grade', 'last_prediction']])
        
        print("Train DCGs")
        lambdas_per_query['discounted_gain'] = lambdas_per_query['gain'] * lambdas_per_query['discount'] 
        dcg = lambdas_per_query[lambdas_per_query['display_rank'] < 10].groupby('qid')['discounted_gain'].sum().mean()
        print("mean   ", dcg)
        print("----------")
        
        lambdas_per_query = lambdas_per_query.drop(['lambda', 'weight'], axis=1)
    return lambdas_per_query


lambdas_per_query = lambda_mart_pure(judgments=judgments, rounds=50, max_leaf_nodes=10, learning_rate=0.01)

NameError: ignored